In [16]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

import astropy.cosmology as cosmo
import astropy.units as u
import astropy.constants as const

from astropy.table import Table

from sedpy.observate import load_filters

import os
import h5py

from prospect.sources.galaxy_basis import CSPSpecBasis, FastStepBasis
from prospect.observation import Photometry, Spectrum, PolyOptCal
from prospect.models.templates import TemplateLibrary
from prospect.models.sedmodel import SpecModel
from prospect.models import priors
from prospect.likelihood import NoiseModelCov
from prospect.likelihood.kernels import Uncorrelated

from prospect.fitting import fit_model, lnprobfn
from prospect.io import write_results as writer


In [17]:
# Functions for loading in data
@u.quantity_input
def convert_janksy_to_maggies(flux_jy: u.Jy):
    flux_maggie = (flux_jy / (3631 * u.Jy)).decompose().value
    return flux_maggie

def load_prism_data(dir_name, name, version, extra_nod, units):

    # Open file
    file_name = f'{name}_prism_clear_v{version:.1f}_{extra_nod}_1D.fits'
    file_path = os.path.join(dir_name, file_name)
    hdu = fits.open(file_path)

    # Access spectral data
    wave = hdu['WAVELENGTH'].data
    flux = hdu['DATA'].data
    err = hdu['ERR'].data

    # Assign units
    wave = wave * u.m
    flux = flux * u.Unit('W / m3')
    err = err * u.Unit('W / m3')

    # Convert units
    if units == 'original':
        pass
    elif units == 'cgs':
        wave = wave.to(u.um)
        flux = flux.to(u.erg/(u.s * u.cm**2 * u.AA))
        err = err.to(u.erg/(u.s * u.cm**2 * u.AA))
    elif units == 'jy':
        wave = wave.to(u.um)
        # flux_cgs = flux.to(u.erg/(u.s * u.cm**2 * u.AA))
        # err_cgs = err.to(u.erg/(u.s * u.cm**2 * u.AA))
        # flux = flux_cgs.to(u.Jy)
        # err = err_cgs.to(u.Jy)
        flux = flux.to(u.Jy, equivalencies=u.spectral_density(wave))
        err = err.to(u.Jy, equivalencies=u.spectral_density(wave))
    elif units == 'maggies':
        wave = wave.to(u.AA)
        flux_jy = flux.to(u.Jy, equivalencies=u.spectral_density(wave))
        err_jy = err.to(u.Jy, equivalencies=u.spectral_density(wave))
        flux = convert_janksy_to_maggies(flux_jy)
        err = convert_janksy_to_maggies(err_jy)

    return wave, flux, err

def load_photometry_data(dir_name, name, units):
    

In [ ]:
# Functions for building model
def zred_to_agebins(zred=None, nbins_sfh=None, **extras):
        """ Returns age bins going [0, 10Myr, 30Myr, 100Myr, ...] and nbin-2 equally spaced (in logtime) bins from 100Myr to age_universe
        """

        tuniv = np.squeeze(cosmo.age(zred).to("yr").value)
        ncomp = np.squeeze(nbins_sfh)
        tbinmax = np.squeeze(cosmo.age(20).to("yr").value)
        tbinmax = tuniv - tbinmax
        #agelims = [0.0, 7.4772] + np.linspace(8.0, np.log10(tbinmax), ncomp-1).tolist()
        logtmax = np.log10(2*10**9)
        agelims = [0.0, 7.0, 7.4772] \
                    + np.linspace(8.0, 9.0, ncomp-6, endpoint=False).tolist() \
                    + np.log10(np.linspace(1.*10**9, tbinmax, 4, endpoint=True)).tolist()
        agebins = np.array([agelims[:-1], agelims[1:]])
        return agebins.T

def logmass_to_masses(logmass=None, logsfr_ratios=None, zred=None, **extras):
        """ Computes masses formed in each bin from SFR ratios
        """
        agebins = zred_to_agebins(zred=zred, **extras)
        logsfr_ratios = np.clip(logsfr_ratios, -10, 10)  # numerical issues...
        nbins = agebins.shape[0]
        sfr_ratios = 10**logsfr_ratios
        dt = (10**agebins[:, 1] - 10**agebins[:, 0])
        coeffs = np.array([(1./np.prod(sfr_ratios[:i])) * (np.prod(dt[1:i+1]) / np.prod(dt[:i])) for i in range(nbins)])
        m1 = (10**logmass) / coeffs.sum()
        return m1 * coeffs

def to_dust1(dust1_fraction=None, dust1=None, dust2=None, **extras):
        return dust1_fraction*dust2

In [18]:
class PolySpectrum(PolyOptCal, Spectrum):
    pass

# Define observations
def build_obs(obs_params):
    """Build a set of Prospector observations using the `Prospector.prospect.observation.Observation` class

    Parameters
    ----------
    obs_params : dict
        input arguments needed to extract wavelength, flix and uncertainty information and convert to the correct units

    Returns
    -------
    obs : list
        list of `Prospector.prospect.observation.Observation` classes made up of spectra and photometry
    """

    # Load prism data
    pwave, pflux, perr = load_prism_data(**obs_params)

    # Create Spectrum and Photometry classes
    # -- prism spectrum
    spec = Spectrum(wavelength=pwave, flux=pflux, 
                    uncertainty=perr)
    polyspec = PolySpectrum(wavelength=pwave, flux=pflux, 
                    uncertainty=perr, polynomial_order=7)  # optimise polynomial spectral calibration (or set as prior in model)

    # -- medium-grating spectrum
    # TODO: Add medium grating spectra
    # -- nircam photometry
    # TODO: Add nircam photometry values
    phot = Photometry(filters=None, flux=None,
                       uncertainty=None, mask=None)  # no photometry

    # Build obs from spectrum and photometry
    # -- ensures all required keys are present for fitting
    spec.rectify()
    phot.rectify()
    obs = [spec, phot]

    return obs

In [19]:
def build_model(model_kwargs):
    """Build a `Prospector.models.sedmodel.SpecModel` class using a `ProspectorParams` object

    Parameters
    ----------
    models_kwargs : dict
        kwargs for building the model (e.g., turn nebular meission on/off)

    Returns
    -------
    model : `Prospector.models.sedmodel.SpecModel`
        model to be fit to data
    """
    
    # Continuity SFH
    model_params = TemplateLibrary["continuity_sfh"]

    # Add nebular emission
    if model_kwargs['add_nebular']:
        model_params.update(TemplateLibrary["nebular"])
        model_params['gas_logu']['init'] = -2.0
        model_params['gas_logu']['isfree'] = True
        model_params['gas_logz']['isfree'] = True

        # Adjust for widths of emission lines
        model_params["nebemlineinspec"]["init"] = True
        model_params["eline_sigma"] = dict(N=1, isfree=True, init=100.0, units='km/s', 
                                           prior=priors.TopHat(mini=30, maxi=550))
        
    # Set zred to free
    model_params["zred"]["isfree"] = False
    model_params["zred"]["init"] = model_kwargs['zred']
    model_params["zred"]["prior"] = priors.TopHat(mini=3.0, maxi=3.4)

    # Set IMF
    model_params['imf_type']['init'] = 2 # Kroupa IMF

    # Set SFH prior
    # -- fix number of SFH bins
    nbins_sfh = 9
    model_params["nbins_sfh"] = dict(N=1, isfree=False, init=nbins_sfh)
    model_params['agebins']['N'] = nbins_sfh
    model_params['mass']['N'] = nbins_sfh
    model_params['logsfr_ratios']['N'] = nbins_sfh - 1  # 
    # -- set logSFR bin ratios
    model_params['logsfr_ratios']['init'] = np.full(nbins_sfh - 1, 0.0)  # logSFR = 0 means constant SFH
    model_params['logsfr_ratios']['prior'] = priors.StudentT(mean=np.full(nbins_sfh-1, 0.0),
                                                             scale=np.full(nbins_sfh-1, 0.3),
                                                             df=np.full(nbins_sfh-1, 2))  # use Student's t-distribution parameters from Leja et al. (2019)

    # Scale agebins for redshift such that t_max = t_univ
    model_params['agebins']['depends_on'] = zred_to_agebins

    # Set total mass formed prior
    model_params["logmass"]["isfree"] = True
    model_params["logmass"]["prior"] = priors.TopHat(mini=7, maxi=12)

    # Set mass formed in each bin ---
    model_params["mass"]["isfree"] = False
    model_params['mass']['depends_on'] = logmass_to_masses

    # Set metallicity prior
    model_params["logzsol"]["init"] = np.log10(1.)
    model_params["logzsol"]["prior"] = priors.TopHat(mini=-2., maxi=0.19)
    model_params["logzsol"]["isfree"] = True

    # Complexify dust attenuation
    # -- switch to Kriek and Conroy (2013) dust attenuation
    model_params['dust_type']['init'] = 4
    # -- slope of the (diffuse) attenuation curve, expressed as the index of the power-law that modifies the base Kriek & Conroy/Calzetti shape.
    # -- a value of zero is basically calzetti with a 2175AA bump
    model_params["dust_index"] = {'N': 1, 'isfree': True,
                                  'init': 0.0, 'prior': priors.TopHat(mini=-1.0, maxi=0.2)}
    # -- set attenuation of old stellar light (not birth cloud component)
    model_params["dust2"]["prior"] = priors.ClippedNormal(mini=0.0, maxi=2.0, mean=0.3, sigma=1)
    model_params["dust2"]['isfree'] = True

    # set attenuation due to birth clouds (fitted as a fraction of diffuse component)
    model_params['dust1'] = dict(N=1, isfree=False, init=0,
                                prior=None, depends_on=to_dust1)
    model_params['dust1_fraction'] = dict(N=1, isfree=True, init=1.0,
            prior=priors.ClippedNormal(mini=0.0, maxi=2.0, mean=1.0, sigma=0.3))

    # Set spectral calibration polynomial coefficient priors
    model_params.update(TemplateLibrary['optimize_speccal'])
    model_params["spec_norm"] = {'N': 1, 'isfree': True, 'init': 1.0, 
                                 'units': 'f_true/f_obs', 'prior': priors.Normal(mean=1.0, sigma=0.1)}
    model_params["polyorder"]["init"] = 7  # order of polynomial that's fit to spectrum

    # Pixel outlier models
    model_params['nsigma_outlier_spec'] = dict(N=1, isfree=False, init=50.)
    model_params['f_outlier_spec'] = dict(N=1, isfree=True, init=1e-3,
                                          prior=priors.TopHat(mini=1e-5, maxi=0.01))

    # This is a multiplicative noise inflation term. It inflates the noise in
    # all spectroscopic pixels as necessary to get a statistically acceptable fit.
    model_params['spec_jitter'] = dict(N=1, isfree=True, init=1.0, 
                                       prior=priors.TopHat(mini=0.5, maxi=5.0))
    
    # Build model
    model = SpecModel(model_params)

    return model

In [20]:
def build_sps(zcontinuous=1, **extras):
    """Build an SPS object
    :param zcontinuous: (default: 1)
        python-fsps parameter controlling how metallicity interpolation of the
        SSPs is acheived.  A value of `1` is recommended.
        * 0: use discrete indices (controlled by parameter "zmet")
        * 1: linearly interpolate in log Z/Z_sun to the target metallicity
             (the parameter "logzsol".)
        * 2: convolve with a metallicity distribution function at each age.
             The MDF is controlled by the parameter "pmetals"
    """
    
    sps = FastStepBasis(zcontinuous=zcontinuous, compute_vega_mags=False)
    
    return sps

In [21]:
def build_noise(add_jitter=False, **extras):
    if add_jitter:
        jitter = Uncorrelated(parnames=['spec_jitter'])
        spec_noise = NoiseModelCov(kernels=[jitter], metric_name='unc', weight_by=['unc'])
        return spec_noise
    else:
        return None

In [34]:
def build_all(obs_params, model_kwargs, noise_kwargs):

    obs = build_obs(obs_params)
    model = build_model(**model_kwargs)
    sps = build_sps()
    noise = build_noise(**noise_kwargs)

    return obs, model, sps, noise


In [35]:
def main():

    obs_params = {
    'dir_name' : '/Users/Jonah/PhD/Research/quiescent_galaxies/data_processed/turner+25_example',
    'name' : '007329',
    # 'name' : '013079',
    'version' : 3.1,
    # 'version' : 1.1,
    'extra_nod' : 'extr5',
    'units' : 'maggies',
    }
    model_kwargs = {
        'add_neb' : True
    }
    noise_kwargs = {
        'add_jitter' : True
    }

    # Load all
    obs, model, sps, noise = build_all(obs_params, model_kwargs, noise_kwargs)

    print("obs:", obs)
    print("model", model)
    print("sps:", sps)

    # Limit observations to spectra
    spec_obs = [obs[0]]

    # Run minimization ----
    run_params = {}
    run_params["dynesty"] = False
    run_params["emcee"] = False
    run_params["optimize"] = True
    run_params["min_method"] = 'lm'

    output = fit_model(spec_obs, model, sps, lnprobfn=lnprobfn, **run_params)

    print(output)

In [36]:
main()

TypeError: only dimensionless scalar quantities can be converted to Python scalars